In [1]:
import sys
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import datetime
import os
import findspark

date = '2022-05-25'
days_count = 7
events_base_path    = '/user/inthemidde/data/events'
interests_base_path = '/user/inthemidde/data/analytics/connection_interests_d28'
verified_tags_path  = '/user/master/data/snapshots/tags_verified/actual'
output_base_path    = '/user/inthemidde/data/analytics'

os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'
findspark.init()
findspark.find()

conf = SparkConf().setAppName(f"ConnectionInterestsJob-{date}-d{days_count}")
sc = SparkContext(conf=conf)
spark = SQLContext(sc)
print(f'spark={spark}')

23/08/16 18:32:18 WARN Utils: Your hostname, fhmflc47vv33sqkc2l5i resolves to a loopback address: 127.0.1.1; using 172.16.0.29 instead (on interface eth0)
23/08/16 18:32:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/16 18:32:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


spark=<pyspark.sql.context.SQLContext object at 0x7fe65b77aa90>


/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
 messages = spark.read.parquet('hdfs:///user/inthemidde/data/events/date=2022-05-25')\
        .where("event_type='message'")

In [4]:
direct_messages = messages.where("event.message_to is not null")
posts = messages.where("event.message_channel_to is not null")
print(direct_messages.count(), posts.count())

476 433


In [8]:
interests = spark.read.parquet(f"hdfs:///user/inthemidde/data/analytics/user_interests_05_31_28")
print(interests.count())

169860


In [9]:
subscriptions = spark.read.parquet('hdfs:///user/inthemidde/data/events')\
        .where(f"event_type = 'subscription' and date <= '{date}'")
print(subscriptions.count())

25034646


In [12]:
verified_tags = spark.read.parquet(f'hdfs://{verified_tags_path}')
print(verified_tags.count())

1000


In [13]:
def get_contacts(direct_messages):
    return direct_messages\
    .select(F.col("event.message_from").alias("from"),
        F.col("event.message_to").alias("to"),
        F.explode(F.array(F.col("event.message_from"), F.col("event.message_to"))).alias("user_id"))\
    .withColumn("contact_id", F.when(F.col("user_id") == F.col("from"), F.col("to")).otherwise(F.col("from")))\
    .select("user_id", "contact_id")\
    .distinct()

In [14]:
contacts = get_contacts(direct_messages)
print(contacts.count())

918


In [15]:
def get_contact_interests(contacts, interests):
    return contacts\
    .withColumnRenamed("user_id", "u")\
    .join(interests, F.col("contact_id") == F.col("user_id"))\
    .transform(lambda df: add_tag_usage_count(df, "u"))\
    .transform(lambda df: add_tag_rank(df, "u"))\
    .groupBy("u")\
    .agg(*[top_direct_tag(c) for c in result_columns()])\
    .withColumnRenamed("u", "user_id")

In [18]:
def tag_columns(reaction):
    return [f"{reaction}_tag_top_{X}" for X in range(1, 4)]
 
def result_columns():
    return tag_columns("like") + tag_columns("dislike") 

def add_tag_usage_count(df, key):
    res = df
    cols = result_columns()
    for c in cols:
        res = res.withColumn(c+'_count', F.count(c).over(Window.partitionBy(key, c)))
    return res 

def add_tag_rank(df, key):
    res = df
    for c in result_columns():
        res = res.withColumn(c+"_rank", F.row_number().over(Window.partitionBy(key).orderBy(F.desc(c), F.desc(c+"_count"))))
    return res
 
def top_direct_tag(column):
    return F.first(F.when(F.col(column+"_rank") == 1, F.col(column)), True).alias('direct_'+column)

contact_interests = get_contact_interests(contacts, interests).cache()
print(contact_interests.count())

581


In [19]:
def get_subs_interests(posts, subscriptions, verified_tags):
    post_tags = posts\
    .select(F.col("event.message_channel_to").alias("channel_id"),
    F.col("event.tags").alias("tags"))
 
    verified_sub_tags = post_tags\
    .join(subscriptions,
    (F.col("event.subscription_channel") == F.col("channel_id")))\
    .select(F.col("event.user").alias("user_id"), F.explode(F.col("tags")).alias("tag"))\
    .join(verified_tags, "tag", "left_semi")
 
    return verified_sub_tags\
    .groupBy("user_id", "tag")\
    .agg(F.countDistinct("*").alias("tag_count"))\
    .withColumn("rank", F.row_number().over(Window.partitionBy("user_id").orderBy(F.desc("tag_count"), F.desc("tag"))))\
    .where("rank <= 3")\
    .groupBy("user_id")\
    .pivot("rank", [1, 2, 3])\
    .agg(F.first("tag"))\
    .withColumnRenamed("1", "sub_verified_tag_top_1")\
    .withColumnRenamed("2", "sub_verified_tag_top_2")\
    .withColumnRenamed("3", "sub_verified_tag_top_3")

subs_interests = get_subs_interests(posts, subscriptions, verified_tags).cache()
print(subs_interests.count())

0


In [31]:
def join_result(contact_interests, subs_interests):
    return contact_interests\
        .join(subs_interests, "user_id", "full_outer")

result = join_result(contact_interests, subs_interests)
print(result.count())

581


In [32]:
for i in range(1,5,1):
    result = result.union(result)
print(result.count())
result = result.limit(9067)
result.write.mode("overwrite").parquet(f"/user/inthemidde/data/analytics/connection_interests_d7/date=2022-05-25")

9296
